In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/thesis

In [ ]:
# !pip install igraph
# !pip install scikit-network

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ------------------------------ --------- 2.1/2.7 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 9.3 MB/s eta 0:00:00


In [ ]:
# !pip install torchmetrics

   ---------------------------------------- 0.0/983.0 kB ? eta -:--:--
   ---------------------------------------- 983.0/983.0 kB 48.1 MB/s  0:00:00

   ---------------------------------------- 0/2 [lightning-utilities]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- ------------------- 1/2 [torchmetrics]
   -------------------- 

In [ ]:
#!conda install pytorch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 cpuonly -c pytorch

^C


In [ ]:
# import torch
# import os
# print("PyTorch has version {}".format(torch.__version__))

# srs_url = f"https://pytorch-geometric.com/whl/torch-{torch.__version__}.html"

PyTorch has version 2.1.0


In [ ]:
# !pip install torch_scatter -f $srs_url
# !pip install torch_sparse -f $srs_url
# !pip install torch_cluster -f $srs_url
# !pip install torch_spline_conv -f $srs_url

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ---------------------------------------- 0.0/788.9 kB ? eta -:--:--
     ------------------------------------- 788.9/788.9 kB 17.0 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html


In [ ]:
# !pip install torch_geometric

  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

   ----------------------------------------  0/13 [tqdm]
   ------ ---------------------------------  2/13 [propcache]
   ------------ ---------------------------  4/13 [fsspec]
   ------------ ---------------------------  4/13 [fsspec]
   --------------- ------------------------  5/13 [frozenlist]
   --------------------- ------------------  7/13 [async-timeout]
   -------------------

In [ ]:
# import torch
# import os
# print("PyTorch has version {}".format(torch.__version__))

# if 'IS_GRADESCOPE_ENV' not in os.environ:
#   !pip install torch==2.4.0

# # Install torch geometric
# if 'IS_GRADESCOPE_ENV' not in os.environ:
#   torch_version = str(torch.__version__)
#   scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
#   sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
#   !pip install torch-scatter -f $scatter_src
#   !pip install torch-sparse -f $sparse_src
#   !pip install torch-geometric
#   !pip install ogb

In [1]:

import re
import os
import sys
import argparse
import yaml
from pprint import pprint
import time
from collections import Counter, defaultdict


from abc import ABC, abstractmethod
import scipy.sparse as sp
from typing import Literal
from dataclasses import dataclass
from typing import List

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import trange

#%env NX_CUGRAPH_AUTOCONFIG=True
import networkx as nx
from networkx.algorithms import bipartite, community
nx.config.warnings_to_ignore.add("cache")

import igraph as ig

import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.data import HeteroData
from torch_geometric.nn import GCNConv, SAGEConv, VGAE, to_hetero
from torch_geometric.utils import train_test_split_edges
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.sampler  import NegativeSampling

from sklearn.cluster import KMeans

from torch_scatter import scatter_mean

In [4]:
x = torch.tensor([[1., 2., 3., 4., 5., 6.],
                   [7., 8., 9., 10., 11., 12.]])   # values
group = torch.tensor([0, 0, 1, 1, 1, 0])     # group ids
out = scatter_mean(x, group, dim=1)          # mean per group
out

tensor([[ 3.,  4.],
        [ 9., 10.]])

In [48]:
parser = argparse.ArgumentParser(description="Load parquet datasets")
parser.add_argument("--config", type=str, default="config.yaml", help="Path to YAML config file")

args, unknown = parser.parse_known_args()

# Load YAML
with open(args.config, "r") as f:
    cfg = yaml.safe_load(f)

In [83]:


def data_processing():
    """
    Load and preprocess the data for the graph neural network.
    An important step is the reindexing of user and book IDs after the filtering process.
    """
    # Load files
    users = pl.read_parquet(os.path.join(DIR, user_filename))
    books = pl.read_parquet(os.path.join(DIR, book_filename))
    interactions = pl.read_parquet(os.path.join(DIR, interactions_filename))
    embeddings_descriptions = pl.read_parquet(os.path.join(DIR, embeddings_descriptions_filename))
    embeddings_reviews = pl.read_parquet(os.path.join(DIR, embeddings_reviews_filename))
    
    # Drop hashed ID columns since we're going to redefine it
    users = users.drop('user_id_hashed', strict=False)
    books = books.drop('book_id_hashed', strict=False)
    interactions = interactions.drop(['user_id_hashed', 'book_id_hashed'], strict=False)

    print(f"Initial size. Users: {len(users)}, Books: {len(books)}, Total: {len(users) + len(books)}, Edges: {len(interactions)}, Density: {len(interactions) / (len(users) * len(books)):4%}")
    users = users.filter(pl.col('review_coreness') >= review_coreness_k)
    users = users.with_columns((pl.col('user_id').rank()-1).cast(pl.Int64).alias('user_id_hashed'))
    books = books.filter(pl.col('review_coreness') >= review_coreness_k)
    books = books.with_columns((pl.col('book_id').rank()-1).cast(pl.Int64).alias('book_id_hashed'))

    interactions = interactions.join(users.select(["user_id", "user_id_hashed"]), on="user_id", how="inner")
    interactions = interactions.join(books.select(["book_id", "book_id_hashed"]), on="book_id", how="inner")

    print(f"Filtered size. Users: {len(users)}, Books: {len(books)}, Total: {len(users) + len(books)}, Edges: {len(interactions)}, Density: {len(interactions) / (len(users) * len(books)):4%}")

    embeddings_descriptions = embeddings_descriptions.join(books.select(["book_id", "book_id_hashed"]), on="book_id", how="inner")
    embeddings_reviews = embeddings_reviews.join(interactions.select(["book_id", "user_id", "user_id_hashed", "book_id_hashed"]), on=["book_id", "user_id"], how="inner")

    ## Sorting
    users = users.sort('user_id_hashed')
    books = books.sort('book_id_hashed')
    interactions = interactions.sort('user_id_hashed', 'book_id_hashed')
    embeddings_descriptions = embeddings_descriptions.sort('book_id_hashed')
    embeddings_reviews = embeddings_reviews.sort('user_id_hashed', 'book_id_hashed')

    data = {
        "users": users,
        "books": books,
        "interactions": interactions,
        "embeddings_descriptions": embeddings_descriptions,
        "embeddings_reviews": embeddings_reviews
    }

    return data

def graph_edges(data):
    edges = data['interactions'].select(['user_id_hashed', 'book_id_hashed']).rows()
    src, dst = [], []
    for e in edges:
        s, t = e
        src.append(s)
        dst.append(t)
    edge_index = torch.tensor([src, dst], dtype=torch.long)
    return edge_index


def graph_node_random_features(data, n_nodes, dim):
    "Generate random features for graph nodes."
    random_features = nn.Embedding(n_nodes, dim)
    return random_features


def graph_node_topological_features(data, feature: List[str], degree_log_transform=False):
    "Generate topological features for graph nodes."
    if "degree" in feature and degree_log_transform:
        data['users'] = data['users'].with_columns(
            (pl.col('degree').log()).alias('degree')
        )
        data['books'] = data['books'].with_columns(
            (pl.col('degree').log()).alias('degree')
        )

    users = data['users'].sort('user_id_hashed')
    users_features = users.select(feature).to_numpy().T
    books = data['books'].sort('book_id_hashed')
    books_features = books.select(feature).to_numpy().T

    users_feature = torch.tensor(users_features, dtype=torch.float32)
    books_feature = torch.tensor(books_features, dtype=torch.float32)

    return users_feature, books_feature


def graph_one_hot_encoding(n_nodes):
    "Generate one-hot encoding for graph nodes. Not Recommended"
    return torch.eye(n_nodes)


def embeddings_parquet2torch(data):
    """
    Convert embeddings from parquet to torch tensors.
    Embeddings are stored as row, to_numpy operation take the column as numpy row, so we need to transpose them.
    """

    emb_rev_columns = [col for col in data['embeddings_reviews'].columns if col.startswith('column')]
    embeddings_reviews = data['embeddings_reviews'].select(emb_rev_columns)
    embeddings_reviews = embeddings_reviews.to_numpy().T
    embeddings_reviews = torch.tensor(embeddings_reviews, dtype=torch.float32)

    emb_des_columns = [col for col in data['embeddings_descriptions'].columns if col.startswith('column')]
    embeddings_descriptions = data['embeddings_descriptions'].select(emb_des_columns)
    embeddings_descriptions = embeddings_descriptions.to_numpy().T
    embeddings_descriptions = torch.tensor(embeddings_descriptions, dtype=torch.float32)

    assert embeddings_reviews.shape[1] == data['embeddings_reviews'].shape[0] and embeddings_reviews.shape[0] == len(emb_rev_columns), "Mismatch in embeddings_reviews shape"
    assert embeddings_descriptions.shape[1] == data['embeddings_descriptions'].shape[0] and embeddings_descriptions.shape[0] == len(emb_des_columns), "Mismatch in embeddings_descriptions shape"

    return embeddings_reviews, embeddings_descriptions


def graph_node_textual_feature_description(data, embeddings_descriptions):
    "Since the reviews are unique, return the embeddings for the book descriptions."
    return embeddings_descriptions


def graph_node_textual_features_reviews_aggr(data, embeddings_reviews, aggr_fn: Literal["mean", "sum"] = 'mean'):
    "Return the aggregated user review embeddings."
    VALID_AGGR = {"mean", "sum"}
    if aggr_fn not in VALID_AGGR:
        raise ValueError(f"aggr_fn must be 'mean' or 'sum', got '{aggr_fn}'")

    user_ids = data['embeddings_reviews']['user_id_hashed'].to_numpy()
    user_ids = torch.tensor(user_ids, dtype=torch.long)

    book_ids = data['embeddings_reviews']['book_id_hashed'].to_numpy()
    book_ids = torch.tensor(book_ids, dtype=torch.long)

    if aggr_fn == "mean":
        user_review_embeddings_aggr = scatter_mean(embeddings_reviews, user_ids, dim=1)
        book_review_embeddings_aggr = scatter_mean(embeddings_reviews, book_ids, dim=1)
    elif aggr_fn == "sum":
        user_review_embeddings_aggr = scatter_sum(embeddings_reviews, user_ids, dim=1)
        book_review_embeddings_aggr = scatter_sum(embeddings_reviews, book_ids, dim=1)

    return user_review_embeddings_aggr, book_review_embeddings_aggr


def one_strategy(data):
    "One strategy for generating graph node features."
    # Topological Features
    users_topo_feature, books_topo_feature = graph_node_topological_features(data, feature=["degree"], degree_log_transform=True)
    
    # Textual Features
    embeddings_reviews, embeddings_descriptions = embeddings_parquet2torch(data)
    books_textual_feature = graph_node_textual_feature_description(data, embeddings_descriptions)
    user_review_embeddings_aggr, book_review_embeddings_aggr = graph_node_textual_features_reviews_aggr(data, embeddings_reviews, aggr_fn='mean')

    user_x = torch.cat([users_topo_feature, user_review_embeddings_aggr], dim=1)
    book_x = torch.cat([books_topo_feature, books_textual_feature, book_review_embeddings_aggr], dim=1)

    return user_x, book_x

def second_strategy(data, dimensions):
    # Topological Features
    users_topo_feature, books_topo_feature = graph_node_topological_features(data, feature=["degree"], degree_log_transform=True)

    # Random Features
    user_n = data['users'].shape[0]
    book_n = data['books'].shape[0]
    users_random_feature = torch.randn(user_n, dimensions)
    books_random_feature = torch.randn(book_n, dimensions)

    user_x = torch.cat([users_topo_feature, users_random_feature], dim=1)
    book_x = torch.cat([books_topo_feature, books_random_feature], dim=1)

    return user_x, book_x

def third_strategy(data, dimensions):
    user_n = data['users'].shape[0]
    book_n = data['books'].shape[0]
    user_x = torch.randn(user_n, dimensions)
    book_x = torch.randn(book_n, dimensions)

    return user_x, book_x

In [53]:
class GraphDataLoader:
    def __init__(self, config):
        self.cfg = config
    
    def load(self):
        "Load the files by reading the filenames in config"
        data_cfg = self.cfg['data']
        DIR = data_cfg['dir']
        users = pl.read_parquet(os.path.join(DIR, data_cfg['users_filename']))
        books = pl.read_parquet(os.path.join(DIR, data_cfg['books_filename']))
        interactions = pl.read_parquet(os.path.join(DIR, data_cfg['interactions_filename']))
        descriptions = pl.read_parquet(os.path.join(DIR, data_cfg['descriptions_filename']), columns=["book_id", "filtered"])
        reviews = pl.read_parquet(os.path.join(DIR, data_cfg['reviews_filename']), columns=["user_id", "book_id", "filtered"])
        embeddings_descriptions = pl.read_parquet(os.path.join(DIR, data_cfg['embeddings_descriptions_filename']))
        embeddings_reviews = pl.read_parquet(os.path.join(DIR, data_cfg['embeddings_reviews_filename']))
        data =  {
            "users": users,
            "books": books,
            "interactions": interactions,
            "descriptions": descriptions,
            "reviews": reviews,
            "embeddings_descriptions": embeddings_descriptions,
            "embeddings_reviews": embeddings_reviews
        }
        return data

    def preprocess(self, data, method: Literal["coreness", "review_coreness"] = 'review_coreness'):
        "Preprocess the loaded data by eliminating observations based on the method"
        pp_cfg = self.cfg['processing']

        users = data["users"]
        books = data["books"]
        interactions = data["interactions"]
        descriptions = data["descriptions"]
        reviews = data["reviews"]
        embeddings_descriptions = data["embeddings_descriptions"]
        embeddings_reviews = data["embeddings_reviews"]


        # Drop hashed ID columns since we're going to redefine it
        users = users.drop('user_id_hashed', strict=False)
        books = books.drop('book_id_hashed', strict=False)
        interactions = interactions.drop(['user_id_hashed', 'book_id_hashed'], strict=False)

        print(f"Initial size. Users: {len(users)}, Books: {len(books)}, Total: {len(users) + len(books)}, Edges: {len(interactions)}, Density: {len(interactions) / (len(users) * len(books)):4%}")
        
        # Filter by degree. "review_coreness" should be always used
        
        if method == "coreness":
            # Apply coreness filtering
            coreness_k = pp_cfg.get("coreness_k", 3)
            users = users.filter(pl.col('coreness') >= coreness_k)
            books = books.filter(pl.col('coreness') >= coreness_k)
        elif method == "review_coreness":
            # Apply review coreness filtering
            review_coreness_k = pp_cfg.get("review_coreness_k", 3)
            users = users.filter(pl.col('review_coreness') >= review_coreness_k)
            books = books.filter(pl.col('review_coreness') >= review_coreness_k)

        ## Reindexing
        users = users.with_columns((pl.col('user_id').rank()-1).cast(pl.Int64).alias('user_id_hashed'))
        books = books.with_columns((pl.col('book_id').rank()-1).cast(pl.Int64).alias('book_id_hashed'))

        ## Filter and add the new_index information for the rest of data
        interactions = interactions.join(users.select(["user_id", "user_id_hashed"]), on="user_id", how="inner")
        interactions = interactions.join(books.select(["book_id", "book_id_hashed"]), on="book_id", how="inner") # Drop rows with reviews that are not english

        print(f"Filtered size. Users: {len(users)}, Books: {len(books)}, Total: {len(users) + len(books)}, Edges: {len(interactions)}, Density: {len(interactions) / (len(users) * len(books)):4%}")

        embeddings_descriptions = embeddings_descriptions.join(books.select(["book_id", "book_id_hashed"]), on="book_id", how="inner")
        embeddings_reviews = embeddings_reviews.join(interactions.select(["book_id", "user_id", "user_id_hashed", "book_id_hashed"]), on=["book_id", "user_id"], how="inner")

        ## Filter also the embeddings
        print(f"Initial textual embeddings size. Descriptions: {len(embeddings_descriptions)}, Reviews: {len(embeddings_reviews)}, Total: {len(embeddings_descriptions) + len(embeddings_reviews)}")
        descriptions = data['descriptions'].filter(pl.col("filtered") == 1).drop("filtered")
        embeddings_descriptions = embeddings_descriptions.join(descriptions[["book_id"]], on="book_id", how="inner")

        reviews = data['reviews'].filter(pl.col("filtered") == 1).drop("filtered")
        embeddings_reviews = embeddings_reviews.join(reviews[["user_id", "book_id"]], on=["user_id", "book_id"], how="inner")
        print(f"Filtered textual embeddings size. Descriptions: {len(embeddings_descriptions)}, Reviews: {len(embeddings_reviews)}, Total: {len(embeddings_descriptions) + len(embeddings_reviews)}")

        ## Sorting
        users = users.sort('user_id_hashed')
        books = books.sort('book_id_hashed')
        interactions = interactions.sort('user_id_hashed', 'book_id_hashed')
        embeddings_descriptions = embeddings_descriptions.sort('book_id_hashed')
        embeddings_reviews = embeddings_reviews.sort('user_id_hashed', 'book_id_hashed')

        data = {
            "users": users,
            "books": books,
            "interactions": interactions,
            "embeddings_descriptions": embeddings_descriptions,
            "embeddings_reviews": embeddings_reviews
        }

        return data


In [89]:
class TextEmbeddingProcessor:
    def __init__(self, cfg):
        self.cfg = cfg # It won't be used

    def to_torch(self, data):
        if  isinstance(data['embeddings_reviews'], pl.DataFrame):
            return self.embeddings_parquet2torch(data)
        else:
            pass

    def embeddings_parquet2torch(self, data):
        """
        Convert embeddings from parquet to torch tensors.
        Return also the index list for each embeddings 
        """

        # Reviews
        reviews = data['embeddings_reviews']

        ## Extract the embeddings
        emb_rev_columns = [col for col in reviews.columns if col.startswith('column')]
        embeddings_reviews = reviews.select(emb_rev_columns)
        embeddings_reviews = embeddings_reviews.to_numpy()
        embeddings_reviews = torch.tensor(embeddings_reviews, dtype=torch.float32)

        ## Extract the ids
        users_review_id = reviews['user_id_hashed'].to_numpy()
        users_review_id = torch.tensor(users_review_id, dtype=torch.long)
        books_review_id = reviews['book_id_hashed'].to_numpy()
        books_review_id = torch.tensor(books_review_id, dtype=torch.long)

        # Descriptions
        descriptions = data['embeddings_descriptions']

        ## Extract the embeddings
        emb_des_columns = [col for col in descriptions.columns if col.startswith('column')]
        embeddings_descriptions = descriptions.select(emb_des_columns)
        embeddings_descriptions = embeddings_descriptions.to_numpy()
        embeddings_descriptions = torch.tensor(embeddings_descriptions, dtype=torch.float32)

        books_des_id = descriptions['book_id_hashed'].to_numpy()
        books_des_id = torch.tensor(books_des_id, dtype=torch.long)

        assert embeddings_reviews.shape[0] == reviews.shape[0] and embeddings_reviews.shape[1] == len(emb_rev_columns), "Mismatch in embeddings_reviews shape"
        assert embeddings_descriptions.shape[0] == descriptions.shape[0] and embeddings_descriptions.shape[1] == len(emb_des_columns), "Mismatch in embeddings_descriptions shape"

        # Overwrite to free the memory
        data['embeddings_reviews'] = embeddings_reviews
        data['embeddings_descriptions'] = embeddings_descriptions
        data['users_review_id'] = users_review_id
        data['books_review_id'] = books_review_id
        data['books_des_id'] = books_des_id

        return data


In [ ]:
class FeatureExtractor(ABC):
    """Abstract base class for all feature extractors."""

    @abstractmethod
    def build(self, data, node_type: str) -> torch.Tensor:
        """Each extractor must implement this to return a feature tensor."""
        pass

class TopologicalFeatures(FeatureExtractor):
    def __init__(self, features: List[str], degree_log_transform: bool = False):
        self.features = features
        self.degree_log_transform = degree_log_transform

    def build(self, data, node_type: Literal["users", "books"]) -> torch.Tensor:
        df = data[node_type]  # "users" or "books"

        if "degree" in self.features and self.degree_log_transform:
            df = df.with_columns((pl.col("degree").log()).alias("degree"))

        feats = df.select(self.features).to_numpy()
        return torch.tensor(feats, dtype=torch.float32)


class OneHotEncodedFeatures(FeatureExtractor):
    def build(self, data, node_type):
        "Generate one-hot encoding for graph nodes. Not Recommended"
        n_nodes = data[node_type].shape[0]
        return torch.eye(n_nodes)


class RandomFeatures(FeatureExtractor):
    def __init__(self, dim: int):
        self.dim = dim

    def build(self, data, node_type: Literal["users", "books"]) -> torch.Tensor:
        num_nodes = data[node_type].shape[0]
        return nn.Embedding(num_nodes, self.dim)


class TextualDescriptionFeatures(FeatureExtractor):
    def __init__(self):
        pass

    def build(self, data, node_type: Literal["users", "books"]) -> torch.Tensor:
        if node_type == "users":
            print(f"Users do not have descriptions")
            return None
        else:
            ids = data['books_des_id']
            embeddings = data['embeddings_descriptions']
            perm = torch.argsort(ids)
            embeddings = embeddings[perm]
            return embeddings

class TextualReviewFeatures(FeatureExtractor):
    def __init__(self, aggr_fn: Literal["mean", "sum"] = 'mean'):
        VALID_AGGR = {"mean", "sum"}
        if aggr_fn not in VALID_AGGR:
            raise ValueError(f"aggr_fn must be 'mean' or 'sum', got '{aggr_fn}'")
        self.aggr_fn = aggr_fn

    def build(self, data, node_type: Literal["users", "books"]) -> torch.Tensor:

        ids = data[f'{node_type}_review_id']  # 'users_review_id' or 'books_review_id'
        embeddings_reviews = data['embeddings_reviews']

        if self.aggr_fn == "mean":
            embeddings_reviews_aggr = scatter_mean(embeddings_reviews, ids, dim=0)
        elif self.aggr_fn == "sum":
            embeddings_reviews_aggr = scatter_sum(embeddings_reviews, ids, dim=0)

        return embeddings_reviews_aggr


In [103]:
FEATURE_REGISTRY = {
    "topo": TopologicalFeatures,
    "random": RandomFeatures,
    "textual_desc": TextualDescriptionFeatures,
    "textual_reviews": TextualReviewFeatures,
}


class GraphLoader():
    def __init__(self, cfg):
        self.cfg = cfg

    def build_edges(self, data):
        "Build graph edges from the data."
        edges = data['interactions'].select(['user_id_hashed', 'book_id_hashed']).rows()
        src, dst = [], []
        for e in edges:
            s, t = e
            src.append(s)
            dst.append(t)
        edge_index = torch.tensor([src, dst], dtype=torch.long)
        return edge_index

    def build_nodes_features(self, data):
        """Return user_x, book_x feature tensors."""
        user_feats, book_feats = [], []

        for fconf in self.cfg.get("user", []):
            extractor_cls = FEATURE_REGISTRY[fconf["name"]]
            extractor = extractor_cls(**{k: v for k, v in fconf.items() if k != "name"})
            feat = extractor.build(data, "users")
            if feat is not None:
                user_feats.append(feat)

        for fconf in self.cfg.get("book", []):
            extractor_cls = FEATURE_REGISTRY[fconf["name"]]
            extractor = extractor_cls(**{k: v for k, v in fconf.items() if k != "name"})
            feat = extractor.build(data, "books")
            if feat is not None:
                book_feats.append(feat)

        user_x = torch.cat(user_feats, dim=1) if user_feats else None
        book_x = torch.cat(book_feats, dim=1) if book_feats else None

        return user_x, book_x

    def build(self, data):
        edge_index = self.build_edges(data)
        user_x, book_x = self.build_nodes_features(data)
        
        graph_data = HeteroData()
        graph_data['user'].x = user_x
        graph_data['book'].x = book_x
        graph_data[('user', 'interact', 'book')].edge_index = edge_index
        graph_data = T.ToUndirected()(graph_data) # Convert to undirected
        return graph_data



In [92]:
## Scratch pipeline
graph_data_loader = GraphDataLoader(cfg)
text_embedding_processor = TextEmbeddingProcessor(cfg)

data = graph_data_loader.load()
data = graph_data_loader.preprocess(data, method='review_coreness')
data = text_embedding_processor.to_torch(data)

Initial size. Users: 43807, Books: 236065, Total: 279872, Edges: 1134132, Density: 0.010967%
Filtered size. Users: 3305, Books: 3799, Total: 7104, Edges: 164377, Density: 1.309183%
Initial textual embeddings size. Descriptions: 3799, Reviews: 32239, Total: 36038
Filtered textual embeddings size. Descriptions: 3799, Reviews: 30733, Total: 34532


In [81]:
data['embeddings_descriptions'].shape, data['embeddings_reviews'].shape

(torch.Size([3799, 384]), torch.Size([30733, 384]))

In [82]:
ids = data['books_des_id']
embeddings = data['embeddings_descriptions']

sorted_index, perm = torch.sort(ids)

embeddings = embeddings[perm]

In [ ]:
data

In [88]:
ids = data[f'users_review_id']

embeddings_reviews = data['embeddings_reviews']

embeddings_reviews_aggr = scatter_mean(embeddings_reviews, ids, dim=0)
embeddings_reviews_aggr.shape

torch.Size([3305, 384])

In [104]:
graph_loader = GraphLoader(cfg)

graph_data = graph_loader.build(data)

In [105]:
graph_data

HeteroData(
  user={ x=[3305, 385] },
  book={ x=[3799, 769] },
  (user, interact, book)={ edge_index=[2, 164377] },
  (book, rev_interact, user)={ edge_index=[2, 164377] }
)

In [ ]:


# def one_strategy(data):
#     "One strategy for generating graph node features."
#     # Topological Features
#     users_topo_feature, books_topo_feature = graph_node_topological_features(data, feature=["degree"], degree_log_transform=True)
    
#     # Textual Features
#     embeddings_reviews, embeddings_descriptions = embeddings_parquet2torch(data)
#     books_textual_feature = graph_node_textual_feature_description(data, embeddings_descriptions)
#     user_review_embeddings_aggr, book_review_embeddings_aggr = graph_node_textual_features_reviews_aggr(data, embeddings_reviews, aggr_fn='mean')

#     user_x = torch.cat([users_topo_feature, user_review_embeddings_aggr], dim=1)
#     book_x = torch.cat([books_topo_feature, books_textual_feature, book_review_embeddings_aggr], dim=1)

#     return user_x, book_x

# def second_strategy(data, dimensions):
#     # Topological Features
#     users_topo_feature, books_topo_feature = graph_node_topological_features(data, feature=["degree"], degree_log_transform=True)

#     # Random Features
#     user_n = data['users'].shape[0]
#     book_n = data['books'].shape[0]
#     users_random_feature = torch.randn(user_n, dimensions)
#     books_random_feature = torch.randn(book_n, dimensions)

#     user_x = torch.cat([users_topo_feature, users_random_feature], dim=1)
#     book_x = torch.cat([books_topo_feature, books_random_feature], dim=1)

#     return user_x, book_x

# def third_strategy(data, dimensions):
#     user_n = data['users'].shape[0]
#     book_n = data['books'].shape[0]
#     user_x = torch.randn(user_n, dimensions)
#     book_x = torch.randn(book_n, dimensions)

#     return user_x, book_x


In [3]:

def igraph_to_hetero_pyg(g, user_type=0, book_type=1, user_node_type='user', item_node_type='item', edge_type='interacts'):
    """
    Convert an igraph bipartite graph to a PyG HeteroData object.
    Assumes:
      - Vertex attribute 'type' distinguishes users (user_type) and books (book_type)
      - Edges are user->book (bipartite)
    """
    # Map igraph node indices to user/item indices
    user_indices = [v.index for v in g.vs if v['type'] == user_type]
    book_indices = [v.index for v in g.vs if v['type'] == book_type]
    user_map = {idx: i for i, idx in enumerate(user_indices)}
    book_map = {idx: i for i, idx in enumerate(book_indices)}

    # Node features (optional: here just zeros)
    user_x = torch.zeros((len(user_indices), 1), dtype=torch.float)
    book_x = torch.zeros((len(book_indices), 1), dtype=torch.float)

    # Edges: only user->book
    src, dst = [], []
    for e in g.es:
        s, t = e.source, e.target
        if g.vs[s]['type'] == user_type and g.vs[t]['type'] == book_type:
            src.append(user_map[s])
            dst.append(book_map[t])
    edge_index = torch.tensor([src, dst], dtype=torch.long)

    data = HeteroData()
    data[user_node_type].x = user_x
    data[item_node_type].x = book_x
    data[(user_node_type, edge_type, item_node_type)].edge_index = edge_index
    data = T.ToUndirected()(data) # Convert to undirected
    return data

# Example usage:
data = igraph_to_hetero_pyg(subg)
print(data)

HeteroData(
  user={ x=[3305, 1] },
  item={ x=[3799, 1] },
  (user, interacts, item)={ edge_index=[2, 164377] },
  (item, rev_interacts, user)={ edge_index=[2, 164377] }
)


In [4]:
def random_node_feature_embedding(data, hidden_channel):
    user_embedding = nn.Embedding(data["user"].num_nodes, hidden_channel)
    item_embedding = nn.Embedding(data["item"].num_nodes, hidden_channel)
    return user_embedding, item_embedding

hidden_channel = 64

user_embedding, item_embedding = random_node_feature_embedding(data, hidden_channel)

data['user'].x = user_embedding.weight
data['item'].x = item_embedding.weight

In [50]:
nn.Embedding(30, 64).weight.shape

torch.Size([30, 64])

In [ ]:
# import torch
# from torch_geometric.data import Data
# import xml.etree.ElementTree as ET

# def read_graphml_to_hetero_pyg(path, user_prefix="u", item_prefix="i"):
#     tree = ET.parse(path)
#     root = tree.getroot()
#     ns = {'g': 'http://graphml.graphdrawing.org/xmlns'}

#     key_map = {}
#     for key in root.findall('g:key', ns):
#         key_id = key.attrib['id']
#         key_map[key_id] = key.attrib.get('attr.name', key_id)

#     # Separate users and items by prefix
#     user_map, item_map = {}, {}
#     user_features, item_features = [], []
#     for idx, node in enumerate(root.findall('.//g:node', ns)):
#         node_id = node.attrib['id']
#         feat = []
#         for data in node.findall('g:data', ns):
#             value = data.text
#             try:
#                 feat.append(float(value))
#             except (ValueError, TypeError):
#                 pass
#         feat = feat or [0.0]
#         if node_id.startswith(user_prefix):
#             user_map[node_id] = len(user_map)
#             user_features.append(feat)
#         elif node_id.startswith(item_prefix):
#             item_map[node_id] = len(item_map)
#             item_features.append(feat)

#     # Edges: from users to items
#     edge_index = [[], []]
#     edge_attrs = []
#     for edge in root.findall('.//g:edge', ns):
#         src_id = edge.attrib['source']
#         tgt_id = edge.attrib['target']
#         # Only keep user->item edges
#         if src_id in user_map and tgt_id in item_map:
#             src = user_map[src_id]
#             tgt = item_map[tgt_id]
#             edge_index[0].append(src)
#             edge_index[1].append(tgt)
#             feat = []
#             for data in edge.findall('g:data', ns):
#                 value = data.text
#                 try:
#                     feat.append(float(value))
#                 except (ValueError, TypeError):
#                     pass
#             edge_attrs.append(feat or [0.0])

#     # Convert to tensors
#     user_x = torch.tensor(user_features, dtype=torch.float) if user_features else torch.empty((0, 1))
#     item_x = torch.tensor(item_features, dtype=torch.float) if item_features else torch.empty((0, 1))
#     edge_index = torch.tensor(edge_index, dtype=torch.long) if edge_index[0] else torch.empty((2, 0), dtype=torch.long)
#     edge_attr = torch.tensor(edge_attrs, dtype=torch.float) if edge_attrs else torch.empty((0, 1))

#     data = HeteroData()
#     data['user'].x = user_x
#     data['book'].x = item_x
#     data['user', 'interacts', 'book'].edge_index = edge_index
#     data['user', 'interacts', 'book'].edge_attr = edge_attr
#     return data

# # Example usage
# DIR = 'data_sample'
# data = read_graphml_to_pyg(os.path.join(DIR, "graph.graphml"))
# print(data)

Data(x=[150655, 5], edge_index=[2, 1134132], edge_attr=[1134132, 1])


In [5]:
## edge_index : edges used for the message passing
## edge_label : edges used for the supervised learning task.
## Some edges in edge_label_index might be in edge_index, as they might be used for supervision and message passing

# % of edges used for supervision that are not in the message passing (theoretically)
disjoint_train_ratio = 0.0 # We set it zero for transductive learning

transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=disjoint_train_ratio, 
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("user", "interacts", "item"),
    rev_edge_types=("item", "rev_interacts", "user"),
)
train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)

Training data:
HeteroData(
  user={ x=[3305, 64] },
  item={ x=[3799, 64] },
  (user, interacts, item)={
    edge_index=[2, 131503],
    edge_label=[131503],
    edge_label_index=[2, 131503],
  },
  (item, rev_interacts, user)={ edge_index=[2, 131503] }
)

Validation data:
HeteroData(
  user={ x=[3305, 64] },
  item={ x=[3799, 64] },
  (user, interacts, item)={
    edge_index=[2, 131503],
    edge_label=[49311],
    edge_label_index=[2, 49311],
  },
  (item, rev_interacts, user)={ edge_index=[2, 131503] }
)


In [6]:
def validate_edge_indices(data):
    all_index = set()
    edge_index = data["user", "interacts", "item"].edge_index
    for u, v in zip(edge_index[0], edge_index[1]):
        all_index.add((u.item(), v.item()))
    
    all_label_index = set()
    edge_label_index = data["user", "interacts", "item"].edge_label_index
    for u, v in zip(edge_label_index[0], edge_label_index[1]):
        all_label_index.add((u.item(), v.item()))

    assert len(all_index) == len(edge_index[0])
    assert len(all_label_index) == len(edge_label_index[0])

    intersection = all_index.intersection(all_label_index)
    

    if len(intersection) > 0:
        print(f"Warning: Overlapping edges found between edge_index and edge_label_index")
        print(f"Overlapping edges size: {len(intersection)}")
    



validate_edge_indices(train_data)
validate_edge_indices(val_data)
validate_edge_indices(test_data)

Overlapping edges size: 131503


In [8]:
train_data["user", "interacts", "item"].edge_label_index

tensor([[ 612, 1065,    8,  ...,  603, 1608, 2245],
        [2017, 3440, 1608,  ..., 3726, 3680,  343]])

In [27]:

batch_size = 64
num_negative_sampling_ratio = 1.0

# Define seed edges:
edge_label_index = train_data["user", "interacts", "item"].edge_label_index
edge_label = train_data["user", "interacts", "item"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    edge_label_index=(("user", "interacts", "item"), edge_label_index),
    edge_label=edge_label,
    batch_size=batch_size,
    neg_sampling=NegativeSampling(mode='binary', amount=num_negative_sampling_ratio), # Let's first use the default option, we will improve it later if needed
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "interacts", "item"].edge_label_index.size(1) == (num_negative_sampling_ratio + 1) * batch_size
assert sampled_data["user", "interacts", "item"].edge_label.min() == 0
assert sampled_data["user", "interacts", "item"].edge_label.max() == 1

Sampled mini-batch:
HeteroData(
  user={
    x=[2803, 64],
    n_id=[2803],
  },
  item={
    x=[3018, 64],
    n_id=[3018],
  },
  (user, interacts, item)={
    edge_index=[2, 13499],
    edge_label=[128],
    edge_label_index=[2, 128],
    e_id=[13499],
    input_id=[64],
  },
  (item, rev_interacts, user)={
    edge_index=[2, 14228],
    e_id=[14228],
  }
)


In [ ]:
class VGAE(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder,
                 **kwargs):
        """
        gnn_layer_cls: class of the GNN layer (e.g., GCNConv, SAGEConv)
        encoder_cls: class of the encoder (should accept gnn_layer_cls, in_channels, hidden_channels, latent_dim)
        decoder_cls: class of the decoder (should accept no arguments)
        in_channels: input feature dimension
        hidden_channels: hidden layer dimension
        latent_dim: latent space dimension
        encoder_kwargs: additional kwargs for encoder
        """
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, edge_index):
        mu, logvar = self.encoder(x, edge_index)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def decode(self, z, edge_index):
        return self.decoder(z, edge_index)

    def recon_loss(self, z, pos_edge_index, neg_edge_index):
        pos_score = torch.sigmoid(self.decode(z, pos_edge_index))
        neg_score = torch.sigmoid(self.decode(z, neg_edge_index))
        pos_loss = -torch.log(pos_score + 1e-15).mean()
        neg_loss = -torch.log(1 - neg_score + 1e-15).mean()
        return pos_loss + neg_loss

    def kl_loss(self, mu, logvar):
        kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        return kl_div

class VGAEncoder(nn.Module):
    def __init__(self, GNNLayer, in_channels, hidden_channels, latent_dim):
        """
        in_channels: dimension of the node input features (after projecting items and using user embeddings)
        hidden_channels: hidden units for first GCN layer
        latent_dim: dimension of mu and logvar
        """
        super().__init__()
        self.conv1 = GNNLayer(in_channels, hidden_channels)
        self.conv_mu = GNNLayer(hidden_channels, latent_dim)
        self.conv_logvar = GNNLayer(hidden_channels, latent_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)
        return mu, logvar

class InnerProductDecoder(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, z_user, z_book, edge_index, sigmoid=True):
        edge_feat_user = z_user[edge_index[0]]
        edge_feat_book = z_book[edge_index[1]]
        score = (edge_feat_user * edge_feat_book).sum(dim=-1)
        if sigmoid:
            score = torch.sigmoid(score)
        return score


In [ ]:
# Mini-batch training loop using train_loader and sampled_data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Instantiate model
in_channels = data['user'].x.shape[1]
hidden_channels = 32
latent_dim = 16

encoder = VGAEncoder(GCNConv, in_channels, hidden_channels, latent_dim).to(device)
decoder = InnerProductDecoder().to(device)
model = VGAE(encoder, decoder).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Mini-batch training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm.tqdm(train_loader):
        batch = batch.to(device)
        user_x = batch['user'].x
        book_x = batch['book'].x
        x = torch.cat([user_x, book_x], dim=0)
        edge_index = batch['user', 'interacts', 'book'].edge_index

        # Encode and decode
        z, mu, logvar = model(x, edge_index)

        # Use edge_label_index for positive edges, sample negatives
        pos_edge_index = batch['user', 'interacts', 'book'].edge_label_index
        # Negative sampling: random pairs (for demonstration)
        num_neg = pos_edge_index.shape[1]
        neg_src = torch.randint(0, x.size(0), (num_neg,), device=device)
        neg_dst = torch.randint(0, batch['book'].x.size(0), (num_neg,), device=device)
        neg_edge_index = torch.stack([neg_src, neg_dst], dim=0)

        # Compute losses
        loss_recon = model.recon_loss(z, pos_edge_index, neg_edge_index)
        loss_kl = model.kl_loss(mu, logvar)
        loss = loss_recon + loss_kl

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [ ]:
class VGAE(nn.Module):
    def __init__(self,
                 gnn_layer_cls,
                 encoder,
                 decoder,
                 user_in_channels,
                 item_in_channels,
                 hidden_channels,
                 latent_dim,
                 emb_linear_transform,
                 ):
        """
        gnn_layer_cls: class of the GNN layer (e.g., GCNConv, SAGEConv)
        encoder_cls: class of the encoder (should accept gnn_layer_cls, in_channels, hidden_channels, latent_dim)
        decoder_cls: class of the decoder (should accept no arguments)
        in_channels: input feature dimension
        hidden_channels: hidden layer dimension
        latent_dim: latent space dimension
        encoder_kwargs: additional kwargs for encoder
        """
        super().__init__()
        self.emb_linear_transform = emb_linear_transform
        self.user_linear = nn.Linear(user_in_channels, hidden_channels)
        self.item_linear = nn.Linear(item_in_channels, hidden_channels)

        if user_in_channels != item_in_channels:
            print(f"Inconsistent input feature dimensions: user={user_in_channels}, item={item_in_channels}")
            print(f"Forcing linear transformation")
            self.emb_linear_transform = True

        self.input_channels = hidden_channels if self.emb_linear_transform else user_in_channels

        # Random initialization of embeddings, don't use it for now
        # self.user_embedding = nn.Embedding(data[user].num_nodes, hidden_channels)
        # self.item_embedding = nn.Embedding(data[item].num_nodes, hidden_channels)

        self.encoder = encoder(gnn_layer_cls, hidden_channels, hidden_channels, latent_dim)
        self.encoder = to_hetero(self.encoder, metadata=data.metadata())
        self.decoder = decoder()

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, data):
        if self.emb_linear_transform:
            x_dict = {
                "user": self.user_linear(data["user"].x),
                "item": self.item_linear(data["item"].x)
            }
        else:
            x_dict = {
                "user": data["user"].x,
                "item": data["item"].x
            }

        mu, logvar = self.encoder(x_dict, data.edge_index_dict)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def decode(self, z, edge_index):
        return self.decoder(z, edge_index)

    def recon_loss(self, z, pos_edge_index, neg_edge_index):
        pos_score = self.decode(z, pos_edge_index)
        neg_score = self.decode(z, neg_edge_index)
        pos_loss = -torch.log(pos_score + 1e-15).mean()
        neg_loss = -torch.log(1 - neg_score + 1e-15).mean()
        return pos_loss + neg_loss

    def kl_loss(self, mu, logvar):
        kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        return kl_div

class VGAEncoder(nn.Module):
    def __init__(self, GNNLayer, in_channels, hidden_channels, latent_dim):
        """
        in_channels: dimension of the node input features (after projecting items and using user embeddings)
        hidden_channels: hidden units for first GCN layer
        latent_dim: dimension of mu and logvar
        """
        super().__init__()
        self.conv1 = GNNLayer(in_channels, hidden_channels)
        self.conv_mu = GNNLayer(hidden_channels, latent_dim)
        self.conv_logvar = GNNLayer(hidden_channels, latent_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)
        return mu, logvar

class InnerProductDecoder(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, z_user, z_item, edge_index, sigmoid=True):
        edge_feat_user = z_user[edge_index[0]]
        edge_feat_item = z_item[edge_index[1]]
        score = (edge_feat_user * edge_feat_item).sum(dim=-1)
        if sigmoid:
            score = torch.sigmoid(score)
        return score

    def 

In [34]:
class VGAE(nn.Module):
    def __init__(self,
                 gnn_layer_cls,
                 encoder,
                 decoder,
                 user_in_channels,
                 item_in_channels,
                 hidden_channels,
                 latent_dim,
                 emb_linear_transform,
                 ):
        """
        gnn_layer_cls: class of the GNN layer (e.g., GCNConv, SAGEConv)
        encoder_cls: class of the encoder (should accept gnn_layer_cls, in_channels, hidden_channels, latent_dim)
        decoder_cls: class of the decoder (should accept no arguments)
        in_channels: input feature dimension
        hidden_channels: hidden layer dimension
        latent_dim: latent space dimension
        encoder_kwargs: additional kwargs for encoder
        """
        super().__init__()
        self.emb_linear_transform = emb_linear_transform
        self.user_linear = nn.Linear(user_in_channels, hidden_channels)
        self.item_linear = nn.Linear(item_in_channels, hidden_channels)

        if user_in_channels != item_in_channels:
            print(f"Inconsistent input feature dimensions: user={user_in_channels}, item={item_in_channels}")
            print(f"Forcing linear transformation")
            self.emb_linear_transform = True

        input_channels = hidden_channels if self.emb_linear_transform else user_in_channels

        # Random initialization of embeddings, don't use it for now
        # self.user_embedding = nn.Embedding(data[user].num_nodes, hidden_channels)
        # self.item_embedding = nn.Embedding(data[item].num_nodes, hidden_channels)

        self.encoder = encoder(gnn_layer_cls, input_channels, hidden_channels, latent_dim)
        self.decoder = decoder()

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, data):
        if self.emb_linear_transform:
            user_x = self.user_linear(data["user"].x)
            item_x = self.item_linear(data["item"].x)
        else:
            user_x = data["user"].x
            item_x = data["item"].x

        x = torch.cat([user_x, item_x], dim=0)
        edge_index = data['user', 'interacts', 'item'].edge_index

        mu, logvar = self.encoder(x, edge_index)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def decode(self, z, edge_index):
        return self.decoder(z, edge_index)




class VGAEncoder(nn.Module):
    def __init__(self, GNNLayer, in_channels, hidden_channels, latent_dim):
        """
        in_channels: dimension of the node input features (after projecting items and using user embeddings)
        hidden_channels: hidden units for first GCN layer
        latent_dim: dimension of mu and logvar
        """
        super().__init__()
        self.conv1 = GNNLayer(in_channels, hidden_channels)
        self.conv_mu = GNNLayer(hidden_channels, latent_dim)
        self.conv_logvar = GNNLayer(hidden_channels, latent_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)
        return mu, logvar


class InnerProductDecoder(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, z, edge_index, sigmoid=True):
        edge_feat_user = z[edge_index[0]]
        edge_feat_item = z[edge_index[1]]
        score = (edge_feat_user * edge_feat_item).sum(dim=-1)
        if sigmoid:
            score = torch.sigmoid(score)
        return score
    
    def forward_all(self, z, sigmoid=True):
        "Use it cautiously, as it computes all pairwise interactions"
        A_pred = (z @ z.T).view(-1)
        if sigmoid:
            A_pred = torch.sigmoid(A_pred)
        return A_pred




In [37]:

def binary_recon_loss(z, preds, batch):
    edge_labels = batch["user", "interacts", "item"].edge_label
    pos_mask = edge_labels == 1
    pos_score = preds[pos_mask]
    neg_score = preds[~pos_mask]

    pos_loss = -torch.log(pos_score + 1e-15).mean()
    neg_loss = -torch.log(1 - neg_score + 1e-15).mean()

    return pos_loss + neg_loss

def kl_loss(mu, logvar):
    kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return kl_div

In [47]:
from torchmetrics.classification import AveragePrecision, AUROC

def compute_auc(y_scores, y_true):
    y_true = y_true.long() 
    score = AUROC(task="binary")(y_scores, y_true).item()
    return score

def compute_average_precision(y_scores, y_true):
    y_true = y_true.long() 
    score = AveragePrecision(task="binary")(y_scores, y_true).item()
    return score

def evaluate(loader, model):
    model.eval()
    total_loss = 0
    total_loss_recon = 0
    total_loss_kl = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            z, mu, logvar = model(batch)
            edge_labels = batch["user", "interacts", "item"].edge_label
            edge_index = batch["user", "interacts", "item"].edge_label_index
            preds = model.decode(z, edge_index)
            all_labels.append(edge_labels)
            all_preds.append(preds)

            loss_recon = binary_recon_loss(z, preds, batch)
            loss_kl = kl_loss(mu, logvar)

            loss = loss_recon + loss_kl
            
            total_loss += loss.item()
            total_loss_recon += loss_recon.item()
            total_loss_kl += loss_kl.item()

    y_scores = torch.cat(all_preds)
    y_true = torch.cat(all_labels)
    auc = compute_auc(y_scores, y_true)
    avg_precision = compute_average_precision(y_scores, y_true)
    return { "total_loss": total_loss, 
             "total_loss_recon": total_loss_recon, 
             "total_loss_kl": total_loss_kl, 
             "auc": auc, 
             "average_precision": avg_precision }



In [48]:
# Training loop for your new VGAE version (from cell 17)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

gnn_layer_cls = GCNConv
user_in_channels = train_data["user"].num_features
item_in_channels = train_data["item"].num_features

## Hyperparameters
hidden_channels = 32
latent_dim = 16

# Instantiate model
model = VGAE(
    gnn_layer_cls=gnn_layer_cls,
    encoder=VGAEncoder,
    decoder=InnerProductDecoder,
    user_in_channels=user_in_channels,
    item_in_channels=item_in_channels,
    hidden_channels=hidden_channels,
    latent_dim=latent_dim,
    emb_linear_transform=False
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


epochs = 3

history = {
    "train" : defaultdict(list),
    "val" : defaultdict(list),
}

best_model = None
best_loss = float('inf')

for epoch in trange(epochs, desc="Training", unit="Epochs"):
    total_loss = 0
    total_loss_recon = 0
    total_loss_kl = 0
    model.train()
    
    all_preds = []
    all_labels = []

    for batch in train_loader:

        # Zero gradients
        optimizer.zero_grad()

        batch = batch.to(device)

        z, mu, logvar = model(batch)
        edge_index = batch["user", "interacts", "item"].edge_label_index
        edge_labels = batch["user", "interacts", "item"].edge_label
        preds = model.decode(z, edge_index)

        ## Store the prediction
        all_preds.append(preds)
        all_labels.append(edge_labels)

        ## Compute the loss
        loss_recon = binary_recon_loss(z, preds, batch)
        loss_kl = kl_loss(mu, logvar)
        loss = loss_recon + loss_kl

        # Store the loss
        total_loss += loss.item()
        all_preds.append(preds)
        all_labels.append(edge_labels)

        total_loss_recon += loss_recon.item()
        total_loss_kl += loss_kl.item()
        
        # Loss backward
        loss.backward()
        optimizer.step()

    ## Evaluation
    total_loss /= len(train_loader)
    total_loss_recon /= len(train_loader)
    total_loss_kl /= len(train_loader)
    history["train"]["total_loss"].append(total_loss)
    history["train"]["recon_loss"].append(total_loss_recon)
    history["train"]["kl_loss"].append(total_loss_kl)

    if epoch % 1 == 0:
        # TRAINING DATA
        y_scores, y_true = torch.cat(all_preds), torch.cat(all_labels)
        auc_score = compute_auc(y_scores, y_true)
        avg_precision = compute_average_precision(y_scores, y_true)
        history["train"]["auc"].append(auc_score)
        history["train"]["average_precision"].append(avg_precision)

        print(f"TRAINING DATA")
        print(f"Epoch {epoch}, Total Loss: {total_loss:.4f}, Rec. Loss: {total_loss_recon:.4f}, KL Loss: {total_loss_kl:.4f}")
        print(f"AUC: {auc_score:.4f}, Average Precision: {avg_precision:.4f}")

        # VALIDATION DATA
        val_metrics = evaluate(val_loader, model)
        history["val"]["total_loss"].append(val_metrics["total_loss"])
        history["val"]["recon_loss"].append(val_metrics["total_loss_recon"])
        history["val"]["kl_loss"].append(val_metrics["total_loss_kl"])
        history["val"]["auc"].append(val_metrics["auc"])
        history["val"]["average_precision"].append(val_metrics["average_precision"])

        print(f"VALIDATION DATA")
        print(f"Epoch {epoch}, Total Loss: {val_metrics['total_loss']:.4f}, Rec. Loss: {val_metrics['total_loss_recon']:.4f}, KL Loss: {val_metrics['total_loss_kl']:.4f}")
        print(f"AUC: {val_metrics['auc']:.4f}, Average Precision: {val_metrics['average_precision']:.4f}")



Training:   0%|          | 0/3 [00:00<?, ?Epochs/s]

Training:   0%|          | 0/3 [06:32<?, ?Epochs/s]

TRAINING DATA
Epoch 0, Total Loss: 1.6783, Rec. Loss: 1.4924, KL Loss: 0.1860
AUC: 0.5130, Average Precision: 0.5057


NameError: name 'val_loader' is not defined

In [ ]:
# class NegativeSampling:
#     def __init__(self, num_samples):
#         self.num_samples = num_samples

#     def sample(self, pos_edge_index, num_nodes):
#         neg_src = torch.randint(0, num_nodes, (self.num_samples,))
#         neg_dst = torch.randint(0, num_nodes, (self.num_samples,))
#         neg_edge_index = torch.stack([neg_src, neg_dst], dim=0)
#         return neg_edge_index


In [ ]:
class HybridBPRRatingAutoencoder(nn.Module):
    def __init__(self, num_features_u, num_features_v, hidden_dim, latent_dim, rating_range=(1, 5)):
        super().__init__()

        self.rating_min, self.rating_max = rating_range

        # Encoders (same as before)
        self.encoder_u = nn.Sequential(
            GCNConv(num_features_u, hidden_dim),
            nn.ReLU(),
            GCNConv(hidden_dim, latent_dim)
        )

        self.encoder_v = nn.Sequential(
            GCNConv(num_features_v, hidden_dim),
            nn.ReLU(),
            GCNConv(hidden_dim, latent_dim)
        )

        # Rating prediction head
        self.rating_decoder = nn.Sequential(
            nn.Linear(latent_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def encode(self, x_u, x_v, edge_index):
        z_u = self.encoder_u(x_u, edge_index)
        z_v = self.encoder_v(x_v, edge_index)
        return z_u, z_v

    def predict_score(self, z_u, z_v, user_ids, item_ids):
        """For BPR ranking"""
        user_emb = z_u[user_ids]
        item_emb = z_v[item_ids]
        return torch.sum(user_emb * item_emb, dim=1)

    def predict_rating(self, z_u, z_v, user_ids, item_ids):
        """For explicit rating prediction"""
        user_emb = z_u[user_ids]
        item_emb = z_v[item_ids]
        edge_embeddings = torch.cat([user_emb, item_emb], dim=1)

        raw_ratings = self.rating_decoder(edge_embeddings)
        ratings = raw_ratings * (self.rating_max - self.rating_min) + self.rating_min

        return ratings.squeeze()

    def forward(self, x_u, x_v, edge_index):
        z_u, z_v = self.encode(x_u, x_v, edge_index)
        return z_u, z_v

def hybrid_loss(model, z_u, z_v,
                # For explicit ratings
                rating_user_ids, rating_item_ids, true_ratings, rating_mask,
                # For BPR
                bpr_user_ids, bpr_pos_items, bpr_neg_items,
                rating_weight=1.0, bpr_weight=0.1):
    """
    Combines rating prediction loss and BPR loss
    """
    total_loss = 0

    # Rating loss (only on observed ratings)
    if rating_mask.sum() > 0:
        observed_idx = rating_mask.bool()
        pred_ratings = model.predict_rating(z_u, z_v,
                                           rating_user_ids[observed_idx],
                                           rating_item_ids[observed_idx])
        rating_loss = F.mse_loss(pred_ratings, true_ratings[observed_idx])
        total_loss += rating_weight * rating_loss

    # BPR loss for ranking
    if len(bpr_user_ids) > 0:
        bpr_loss_val = bpr_loss(model, z_u, z_v, bpr_user_ids, bpr_pos_items, bpr_neg_items)
        total_loss += bpr_weight * bpr_loss_val

    return total_loss